## Graph data structure
## Copyright: Jagadeesh Vasudevamurthy
## filename:graph.ipynb¶

# All import here

In [1]:
import sys # For getting Python Version
import os
import enum # For enum
from IPython.display import IFrame
'''
from Graph import *
from GraphType import *
from GraphIO import *
from GraphInOutDir import *
'''
DISPLAYPDFONSCREEN = True #GLOBAL variable. Only on jupyter make it True

# Graph input and output directory setup
# YOU MUST CHANGE 3 lines below

In [2]:
class GraphInOutDir():
    def __init__(self):
        self.graph_files_directory = "/Users/yashjain/Downloads/graphdata/" 
        self.output_directory = "/Users/yashjain/Downloads/graphdata/dot/" 
        ##Note '"
        if (True): #make this to False, if U don't want to install dor
            self.dot2pdf_exe = "/opt/homebrew/bin/dot/"
        else:
            print("YOU DID NOT INSTALL DOT.EXE")
            print("https://dreampuf.github.io/GraphvizOnline/#digraph%20G%20%7B%0A%0A%20%20subgraph%20cluster_0%20%7B%0A%20%20%20%20style%3Dfilled%3B%0A%20%20%20%20color%3Dlightgrey%3B%0A%20%20%20%20node%20%5Bstyle%3Dfilled%2Ccolor%3Dwhite%5D%3B%0A%20%20%20%20a0%20-%3E%20a1%20-%3E%20a2%20-%3E%20a3%3B%0A%20%20%20%20label%20%3D%20%22process%20%231%22%3B%0A%20%20%7D%0A%0A%20%20subgraph%20cluster_1%20%7B%0A%20%20%20%20node%20%5Bstyle%3Dfilled%5D%3B%0A%20%20%20%20b0%20-%3E%20b1%20-%3E%20b2%20-%3E%20b3%3B%0A%20%20%20%20label%20%3D%20%22process%20%232%22%3B%0A%20%20%20%20color%3Dblue%0A%20%20%7D%0A%20%20start%20-%3E%20a0%3B%0A%20%20start%20-%3E%20b0%3B%0A%20%20a1%20-%3E%20b3%3B%0A%20%20b2%20-%3E%20a3%3B%0A%20%20a3%20-%3E%20a0%3B%0A%20%20a3%20-%3E%20end%3B%0A%20%20b3%20-%3E%20end%3B%0A%0A%20%20start%20%5Bshape%3DMdiamond%5D%3B%0A%20%20end%20%5Bshape%3DMsquare%5D%3B%0A%7D");
            print("You need to run manually to get PDF file from dot files")
            self.dot2pdf_exe ="" 
        

    ##################################################################
    # NOTHING CAN BE CHANGED BELOW
    ##################################################################
        
    ###################################################################
    # Calling an dot2pdf executable
    # "C:/Program Files (x86)/Graphviz2.38/bin/dot.exe" -Tpdf 13.dot -o 13.pdf
    ###################################################################
    def execute_dot_2_pdf(self, f:'string'):
        if (len(f) and len(self.dot2pdf_exe)):
            dot_file = f +".dot"
            pdf_file = f +".pdf"
            s = self.dot2pdf_exe + " -Tpdf " + dot_file + " -o " + pdf_file
            print("Executing", s)
            os.system(s)
            

# YOU WRITE CODE HERE
# Depth First serach using TimeStamp

In [3]:
############################################################
# GraphDot.py
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2021
###########################################################

############################################################
# Depth First serach using TimeStamp
###########################################################
class GraphDfsUsingTimeStamp():
    def __init__(self,g:'graph',filename:'string',dfs_order:'list',has_loop:'list of size 1',work:'list of size 1',dfs_traversal_output_file):
        ##NOTHING CAN BE CHANGED HERE
        self._g = g
        self._f = filename
        self._dfs_order = dfs_order
        self._has_loop = has_loop
        self._has_loop[0] = False
        self._work = work
        self._order = []
        self._work[0] = 0
        self._count = 0
        self._dfs_traversal_output_file = dfs_traversal_output_file
        
        ##YOU CAN has any number of private varibles and funcions     
        self.dfsAlgo()
        self.makeDot()
    
        
    def dfsAlgo(self):
        prevValue = -1
        for n in self._g.nodes:
            n.input = 0 
            n.output = 0
        for i in range(self._g.numV()):
            if self._g.nodes[i].num not in self._order:
                self.startDFS(self._g.nodes[i], 1, prevValue)
        self._dfs_order.extend(self._order[::-1])
    
    def startDFS(self, node, counter, prevValue):
        self._work[0]+=1
        self._count+=1
        node.input = self._count
        for e in node.fanout.values():
            if prevValue != e.other:
                if self._g.gtype == GraphType.UNDIRECTED:
                    prevValue = node.num
                nextNode = self.findNode(e.other)
                if nextNode.input > 0 and nextNode.output==0:
                    self._has_loop[0] = True
                    continue
                if nextNode.num in self._order:
                    continue
                self._work[0]+=1
                self.startDFS(self.findNode(e.other), counter + 1, prevValue)
        self._count+=1
        node.output = self._count
        self._order.append(node.num)
        
        
            
    def findNode(self, value):
        for n in self._g.nodes:
            if n.num == value:
                return n
    
    def makeDot(self):
        self._of = open(self._dfs_traversal_output_file,'w')
        print("See dot file at makeDot:",self._dfs_traversal_output_file)
        order = ''
        for i in self._dfs_order:
            order = order + ' ' + self._g.io.get_real_name(i)
        self._of.write('## Yash Jain ####\n## dot -Tpdf'+self._dfs_traversal_output_file+ ' -o '+self._dfs_traversal_output_file.split('.dot')[0]+'.pdf\n');
        hasLoop = 'LOOP'if self._has_loop[0] else 'NOLOOP'
        self._of.write('digraph g {\n label = "['+ order +'] '+hasLoop+'"\n');
        for n in self._g.nodes:
            self._of.write(str(self._g.io.get_real_name(n.num))+'[label = <'+ str(self._g.io.get_real_name(n.num)) + '<BR/><FONT POINT-SIZE="10">'+ str(n.input)+'/'+str(n.output)+'</FONT>>]\n')
        if ( (self._g.gtype == GraphType.UNDIRECTED) or  (self._g.gtype == GraphType.WEIGHTED_UNDIRECTED) ):
            self._of.write("\tedge [dir=none, color=red]\n")
        else:
            self._of.write("\tedge [color=red]\n")
        for node in self._g.nodes: 
            for aedge in node.fanout.values():
                if ( ((self._g.gtype == GraphType.UNDIRECTED) and (node.num < aedge.other)) or (self._g.gtype == GraphType.DIRECTED) ):
                    self._of.write('\t')
                    self._of.write(self._g.io.get_real_name(node.num))
                    self._of.write("->")
                    self._of.write(self._g.io.get_real_name(aedge.other))
                    self._of.write("\n");
                else:
                    if ( ((self._g.gtype == GraphType.WEIGHTED_UNDIRECTED) and (node.num < aedge.other)) or (self._g.gtype == GraphType.WEIGHTED_DIRECTED) ):
                        self._of.write('\t')
                        self._of.write(self._g.io.get_real_name(node.num))
                        self._of.write("->")
                        self._of.write(self._g.io.get_real_name(aedge.other))
                        self._of.write("[label =")
                        self._of.write(str(aedge.cost))
                        self._of.write("]")
                        self._of.write("\n");
        self._of.write("}")
        self._of.close()
        

## NOTHING CAN BE CHANGED BELOW

## Graph Types

In [4]:
class GraphType(enum.Enum): 
    ##NOTHING CAN BE ADDED HERE
    NONE = 0
    UNDIRECTED = 1
    DIRECTED = 2
    WEIGHTED_UNDIRECTED = 3
    WEIGHTED_DIRECTED  = 4

## Input/output of Graph

In [5]:
class GraphIO():
    def __init__(self):
        ##NOTHING CAN BE ADDED HERE
        self._string2int = {} ## Dictionary. Key is string. Value is int in O(1)
        self._int2string = [] ## Vector. Given an int get string in O(1)

    ##########################################################
    # TIME: THETA(1)
    # SPACE: THETA(1)
    ##########################################################
    def insert_or_find(self,name:'string',mustbethere:'bool')->'int':
        if (name in self._string2int):
              return self._string2int.get(name)
        if (mustbethere):
              assert(False)
        #Not in the dict. 
        n = len(self._string2int)
        n1 = len(self._int2string)
        assert(n == n1)
        # Key is string. Value is a number between 0 to n-1. 
        # THETA(1)
        self._string2int[name] = n  
        ## Given a number get String in THETA(1)
        self._int2string.append(name)
        return n
    
    #########################################################
    # TIME: THETA(1)
    # SPACE: THETA(1)
    #########################################################
    def get_real_name(self,n:'int')->'string':
        l = len(self._int2string)
        assert(n >= 0 and n < l)
        ## THETA(1)
        return self._int2string[n]

## Edge class

In [6]:
class Edge():
      def __init__(self, num:'int', cost:'double' = 0.0):
        self.other = num     #Note global. Graph can access
        self.cost = cost   #note global. Graph can access

## Node class

In [7]:
class Node():
    def __init__(self, num:'int'):
        self.num = num     #Note global. Graph can access
        self.fanout = {}   ## Dictionary. Key is int(num). Value is edge 
        self.fanin = {}   ## Dictionary. Key is int(num). Value is edge

    # Does this node has a fanout of 'n'
    # Time: THETA(1) Space; THETA(1)
    def has_a_fanout_edge(self, n:'int') ->'edge' :
        if n in self.fanout.keys(): ##THETA(1) ## Key is int, Value is Edge
            stored_edge = self.fanout[n] # O(1)
            assert(stored_edge.other == n)
            return stored_edge
        return None
      
    # Does this node has a fanin of 'n'
    # Time: THETA(1) Space; THETA(1)
    def has_a_fanin_edge(self, n:'int') ->'edge' :
        if n in self.fanin.keys(): ##THETA(1) ## Key is int, Value is Edge
            stored_edge = self.fanin[n] # O(1)
            assert(stored_edge.other == n)
            return stored_edge
        return None

## Graph class

In [8]:
class Graph():
    def __init__(self, gtype:'GraphType', io:'GraphIO'):
        ##NOTHING CAN BE ADDED HERE
        self.gtype = gtype     #Note public
        self.io = io    # Note public
        self.nodes = []   ## list of nodes. ##Note public
        self._num_edges = 0  #Numner of edges in the graph. For directed E and for Undirected 2E. Note private
        
     
    ##########################################################
    # All public functions. Nothing can be changed below
    ##########################################################
    
    def numV(self):
        return len(self.nodes)
    
    def numE(self):
        return self._num_edges
    
    ##########################################################
    # TIME: THETA(1)
    # SPACE: THETA(1)
    # Nothing can be changed
    ##########################################################
    def build_node_if_not_exist_and_append(self,n:'int')->'Node':
        l = len(self.nodes)
        assert(n >= 0)
        if (n < l):
            return self.nodes[n]
        node = Node(n)
        self.nodes.append(node)
        return self.nodes[n]
    
    ##########################################################
    # TIME: THETA(1)
    # SPACE: THETA(1)
    # Nothing can be changed
    ##########################################################
    def create_edge(self,n1:'Node',n2:'Node',w:'double',fanout:'bool'):
        e = Edge(n2.num,w) #calls constructor of edge
        if (fanout == True):
            ## SEE e is already there in fanouts of node n1
            stored_edge = n1.has_a_fanout_edge(n2.num)
            if (stored_edge):
                v = stored_edge.cost
                if (w < v):
                    stored_edge.cost = w  ##THETA(1)
            else:
                # First time
                assert(e.other == n2.num) 
                n1.fanout[n2.num] = e  ## Key is int, Value is Edge
                self._num_edges = self._num_edges + 1
        else:
            ## SEE e is already there in fanins nodes of n1
            stored_edge = n1.has_a_fanin_edge(n2.num)
            if (stored_edge):
                v = stored_edge.cost
                if (w < v):
                    stored_edge.cost = w  ##THETA(1)
            else:
                # First time
                assert(e.other == n2.num) 
                n1.fanin[n2.num] = e  ## Key is int, Value is Edge
 
    ##########################################################
    # Nothing can be changed
    # TIME: THETA(V + E)
    # SPACE: THETA(V)
    ########################################################## 
    def assert_dfs_passed(self,has_loop:'list of size 1',dfs_order:'list'):
        if ( (self.gtype == GraphType.UNDIRECTED) or  (self.gtype == GraphType.WEIGHTED_UNDIRECTED) ):
            return
        if (has_loop[0] == False):
            v = self.numV()
            visited = []
            for i in range(v):
                visited.append(False)
            for i in dfs_order:
                node = self.nodes[i]
                for aedge in node.fanin.values():
                   assert(visited[aedge.other] == True) 
                visited[node.num] = True
            print("DFS ASSERT PASSED")

    
    ##########################################################
    # TIME: THETA(V + E)
    # SPACE: THETA(V + E)
    ##########################################################
    def build_graph(self, f:'string'):
        b = GraphBuilder(self,f)

    ##########################################################
    # TIME: THETA(V + E)
    # SPACE: THETA(V + E)
    ##########################################################
    def dump(self,name:'string'):
        b = GraphDump(self,name)
        
    ##########################################################
    # TIME: THETA(V + E)
    # SPACE: THETA(1)
    ############################################################
    def print_graph_as_dot_file(self, f:'string'):
        b = GraphDot(self,f)
        
    ##########################################################
    # TIME: THETA(V + E)
    # SPACE: THETA(Largest path)
    ############################################################
    def dfs_using_time_stamp(self, f:'string',dfs_order:'list',has_loop:'list of size 1', work:'list of size 1',dfs_traversal_output_file:'string'):
        b = GraphDfsUsingTimeStamp(self,f,dfs_order,has_loop,work,dfs_traversal_output_file)
    

## Graph Builder class

In [9]:
class GraphBuilder():
    def __init__(self,g:'graph',filename:'string'):
        print(filename, 'filename')
        self._g = g
        self._f = filename
        self._build()
        
    def _build(self):
        notReadline = 0 
        readline = 0 ;
        with open(self._f, "r") as file: 
            data = file.readlines() 
            for aline in data: 
                token = aline.split() 
                size = len(token)
                if ((size < 2) or (size > 3)):
                    notReadline = notReadline + 1
                    print("NOT READ LINE", aline)
                    continue
                readline = readline + 1
                if (size == 2):
                      assert((self._g.gtype == GraphType.UNDIRECTED) or (self._g.gtype == GraphType.DIRECTED))
                else:
                      assert((self._g.gtype == GraphType.WEIGHTED_UNDIRECTED) or (self._g.gtype == GraphType.WEIGHTED_DIRECTED))
                
                ##########################################################
                # WRITE YOUR CODE BELOW
                ##########################################################                
                v1 = self._g.io.insert_or_find(token[0], False);
                v2 = self._g.io.insert_or_find(token[1], False); 
                # We will not allow self loop
                if (v1 != v2):
                    n1 = self._g.build_node_if_not_exist_and_append(v1)
                    n2 = self._g.build_node_if_not_exist_and_append(v2)
                    w = 0
                    if (size == 3):
                        w = float(token[2])
                    ## n1 has a fanout of n2
                    self._g.create_edge(n1,n2,w,True)
                    # n2 has a fanin of n1
                    self._g.create_edge(n2, n1, w, False) #fanin
                    if ( (self._g.gtype == GraphType.UNDIRECTED) or  (self._g.gtype == GraphType.WEIGHTED_UNDIRECTED) ):
                        ## n2 has a fanout of n1
                        self._g.create_edge(n2,n1,w,True)
                        # n1 has a fanin of n2
                        self._g.create_edge(n1, n2,w,False) #fanin


# DUMP GRAPH ON SCREEN AS A TEXT FILE

In [10]:
class GraphDump():
    def __init__(self,g:'graph',name:'string'):
        #NOTHING CAN BE CHANGED HERE
        self._g = g
        self._title = name
        self._dump()

    
    def _dump(self):
        print(self._title)
        ##WRITE YOUR CODE BELOW ##########
        print(self._g.gtype)
        print("Num Vertices =" , self._g.numV())
        print("Num Edges    =" , self._g.numE())
        numedge = 0 ;
        for node in self._g.nodes:
            print(self._g.io.get_real_name(node.num) , "Fanouts: ", end ="") 
            l = len(node.fanout)
            if (l == 0):
                print("NONE")
            else:
                j = 0 ;
                for aedge in node.fanout.values(): ## Key is int, Value is Edge
                    numedge = numedge + 1
                    toname = self._g.io.get_real_name(aedge.other)
                    if (j < l - 1):
                        print(toname , ",", sep ="",  end ="") 
                    else:
                        print(toname)
                    j = j + 1
                    
            print(self._g.io.get_real_name(node.num) , "FanIns: ", end ="") 
            l = len(node.fanin)
            if (l == 0):
                print("NONE")
            else:
                j = 0 ;
                for aedge in node.fanin.values(): ##Key is int, Value is Edge
                    toname = self._g.io.get_real_name(aedge.other)
                    if (j < l - 1):
                        print(toname , ",", sep ="",  end ="") 
                    else:
                        print(toname)
                    j = j + 1       
        assert(numedge == self._g.numE())

## Print Graph as a dot file

In [11]:
class GraphDot():
    def __init__(self,g:'graph',filename:'string'):
        ##NOTHING CAN BE CHANGED HERE
        self._g = g
        self._f = filename
        self._of = open(self._f,'w')
        self._write_dot()
        
    def _write_dot(self):
        print("See dot file at:",self._f)
         ##########################################################
         # WRITE YOUR CODE BELOW
        ########################################################### 
        self._of.write("## Jagadeesh Vasudevamurthy ####\n");
        self._of.write("digraph g {\n");
        if ( (self._g.gtype == GraphType.UNDIRECTED) or  (self._g.gtype == GraphType.WEIGHTED_UNDIRECTED) ):
            self._of.write("\tedge [dir=none, color=red]\n")
        else:
            self._of.write("\tedge [color=red]\n")
        #Time complexity: THETA(V + E)
        for node in self._g.nodes: 
            for aedge in node.fanout.values(): ## Key is int, Value is Edge
                if ( ((self._g.gtype == GraphType.UNDIRECTED) and (node.num < aedge.other)) or (self._g.gtype == GraphType.DIRECTED) ):
                    self._of.write('\t')
                    self._of.write(self._g.io.get_real_name(node.num))
                    self._of.write("->")
                    self._of.write(self._g.io.get_real_name(aedge.other))
                    self._of.write("\n");
                else:
                    if ( ((self._g.gtype == GraphType.WEIGHTED_UNDIRECTED) and (node.num < aedge.other)) or (self._g.gtype == GraphType.WEIGHTED_DIRECTED) ):
                        self._of.write('\t')
                        self._of.write(self._g.io.get_real_name(node.num))
                        self._of.write("->")
                        self._of.write(self._g.io.get_real_name(aedge.other))
                        self._of.write("[label =")
                        self._of.write(str(aedge.cost))
                        self._of.write("]")
                        self._of.write("\n");
        self._of.write("}")

## Build graph from file and print graph as a dot file

In [12]:
class Test_graph_build_and_write_as_dot():
    def __init__(self):
        self._test()
        
    def _test1(self,gname:'graphname',gtype:'GraphType', enodes:'int',eedges:'int'):
        io = GraphIO();
        iodir = GraphInOutDir();
        print(iodir.graph_files_directory)
        full_name = iodir.graph_files_directory + gname +".txt"
        print("Building graph",full_name)
        g = Graph(gtype,io)
        g.build_graph(full_name)
        g.dump(full_name)
        v = g.numV();
        output_file = iodir.output_directory + gname
        g.print_graph_as_dot_file(output_file +".dot")
        if (enodes < 25):
            iodir.execute_dot_2_pdf(output_file)
            pdffile = output_file +".pdf"
            print(pdffile)
            rpath = os.path.relpath(pdffile)
            if (DISPLAYPDFONSCREEN):
                display(IFrame(rpath, width=800, height=400))
        if (v != enodes):
            print("This graph has",enodes,"nodes. But you are telling",v,"nodes")
            assert(v == enodes)
        e = g.numE()
        if (e != eedges):
            print("This graph has",eedges,"edges. But you are telling",e,"edges")
            assert(g.numE() == eedges)
            
        
    def _test(self):
        g = [ ["13",GraphType.UNDIRECTED,7,24],
              ["14",GraphType.WEIGHTED_UNDIRECTED,6,20] ,
              ["15",GraphType.DIRECTED,6,6],
              ["16",GraphType.WEIGHTED_DIRECTED,5,6],
              ["loopparallel",GraphType.WEIGHTED_DIRECTED,4,3],
              ["cat",GraphType.DIRECTED,6,7],
              ["hd2",GraphType.WEIGHTED_DIRECTED,78,1095],
            ] 
        
        for g1 in g:
            self._test1(g1[0],g1[1],g1[2],g1[3]) 
           

# Test DFS implemented using Time stamp

In [13]:
class Test_Dfs_using_time_stamp_alg():
    def __init__(self):
        self._show = False # Change to True for debugging
        #self._test_one()
        self._test()
        
    def _test1(self,gname:'graphname',gtype:'GraphType', expected_has_loop:'Bool'):
        io = GraphIO();
        iodir = GraphInOutDir();
        print(iodir)
        full_name = iodir.graph_files_directory + gname +".txt"
        print("Building graph",full_name)
        g = Graph(gtype,io)
        g.build_graph(full_name)
        if (self._show):
            g.dump(full_name)
        dot_output_file = iodir.output_directory + gname
        g.print_graph_as_dot_file(dot_output_file +".dot")
        iodir.execute_dot_2_pdf(dot_output_file)
        dot_pdffile = dot_output_file +".pdf"
        print(dot_pdffile)
        if (DISPLAYPDFONSCREEN):
            rpath = os.path.relpath(dot_pdffile)
            display(IFrame(rpath, width=800, height=400))
         
        dfs_order = [] #Caller will this array
        has_loop = [False] # List of size 1
        work = [0] # List of size 1
        dfs_dot_output_file = iodir.output_directory + gname +"dfs"
        g.dfs_using_time_stamp(gname,dfs_order,has_loop,work,dfs_dot_output_file + ".dot") 
        iodir.execute_dot_2_pdf(dfs_dot_output_file)

        print("DFS traversal is in")
        print(dfs_dot_output_file + ".pdf")

        print("DFS ORDER: ",end =" ")
        for i in dfs_order:
            print(io.get_real_name(i),end =" ")
        print()
        if (has_loop[0]):
            print("LOOP")
        else:
            print("NOLOOP")
        print("Work Done",work[0])
        assert(has_loop[0] == expected_has_loop)
        g.assert_dfs_passed(has_loop,dfs_order)
        if (DISPLAYPDFONSCREEN):
            rpath = os.path.relpath(dfs_dot_output_file+".pdf")
            display(IFrame(rpath, width=800, height=400))
        
    def _test_one(self):
        g = [ ["1",GraphType.UNDIRECTED,False],
            ] 
        
        for g1 in g:
            self._test1(g1[0],g1[1],g1[2])
        
    def _test(self):
        g = [ 
            ["u1",GraphType.UNDIRECTED,False],
              ["1",GraphType.UNDIRECTED,False] ,
              ["udf1",GraphType.DIRECTED,True] ,
              ["2",GraphType.DIRECTED,False],
              ["3",GraphType.DIRECTED,True],
              ["cat",GraphType.DIRECTED,False],
              ["7",GraphType.WEIGHTED_DIRECTED,False],  
            ] 
       
        for g1 in g:
            self._test1(g1[0],g1[1],g1[2]) 


## main

In [14]:
def main():
    print("Testing Graph starts")
    print(sys.version)
    ops = {
            1: Test_graph_build_and_write_as_dot,
            2: Test_Dfs_using_time_stamp_alg,
    }
    chosen_operation_function = ops.get(2) ## CHANGE 1 depending on assignment given
    chosen_operation_function();
    
    print("Testing Graph ends")

In [15]:
main()

Testing Graph starts
3.9.12 (main, Apr  5 2022, 01:52:34) 
[Clang 12.0.0 ]
Building graph /Users/yashjain/Downloads/graphdata/u1.txt
/Users/yashjain/Downloads/graphdata/u1.txt filename
See dot file at: /Users/yashjain/Downloads/graphdata/dot/u1.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/u1.dot -o /Users/yashjain/Downloads/graphdata/dot/u1.pdf
/Users/yashjain/Downloads/graphdata/dot/u1.pdf


See dot file at makeDot: /Users/yashjain/Downloads/graphdata/dot/u1dfs.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/u1dfs.dot -o /Users/yashjain/Downloads/graphdata/dot/u1dfs.pdf
DFS traversal is in
/Users/yashjain/Downloads/graphdata/dot/u1dfs.pdf
DFS ORDER:  0 1 
NOLOOP
Work Done 3


Building graph /Users/yashjain/Downloads/graphdata/1.txt
/Users/yashjain/Downloads/graphdata/1.txt filename
See dot file at: /Users/yashjain/Downloads/graphdata/dot/1.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/1.dot -o /Users/yashjain/Downloads/graphdata/dot/1.pdf
/Users/yashjain/Downloads/graphdata/dot/1.pdf


See dot file at makeDot: /Users/yashjain/Downloads/graphdata/dot/1dfs.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/1dfs.dot -o /Users/yashjain/Downloads/graphdata/dot/1dfs.pdf
DFS traversal is in
/Users/yashjain/Downloads/graphdata/dot/1dfs.pdf
DFS ORDER:  1 3 5 4 2 
NOLOOP
Work Done 9


Building graph /Users/yashjain/Downloads/graphdata/udf1.txt
/Users/yashjain/Downloads/graphdata/udf1.txt filename
NOT READ LINE 

See dot file at: /Users/yashjain/Downloads/graphdata/dot/udf1.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/udf1.dot -o /Users/yashjain/Downloads/graphdata/dot/udf1.pdf
/Users/yashjain/Downloads/graphdata/dot/udf1.pdf


...

Building graph /Users/yashjain/Downloads/graphdata/cat.txt
/Users/yashjain/Downloads/graphdata/cat.txt filename
NOT READ LINE 

See dot file at: /Users/yashjain/Downloads/graphdata/dot/cat.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/cat.dot -o /Users/yashjain/Downloads/graphdata/dot/cat.pdf
/Users/yashjain/Downloads/graphdata/dot/cat.pdf


See dot file at makeDot: /Users/yashjain/Downloads/graphdata/dot/catdfs.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/catdfs.dot -o /Users/yashjain/Downloads/graphdata/dot/catdfs.pdf
DFS traversal is in
/Users/yashjain/Downloads/graphdata/dot/catdfs.pdf
DFS ORDER:  Cab Cat Mat Car Bar Bat 
NOLOOP
Work Done 10
DFS ASSERT PASSED


Building graph /Users/yashjain/Downloads/graphdata/7.txt
/Users/yashjain/Downloads/graphdata/7.txt filename
NOT READ LINE  
See dot file at: /Users/yashjain/Downloads/graphdata/dot/7.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/7.dot -o /Users/yashjain/Downloads/graphdata/dot/7.pdf
/Users/yashjain/Downloads/graphdata/dot/7.pdf


See dot file at makeDot: /Users/yashjain/Downloads/graphdata/dot/7dfs.dot
Executing /opt/homebrew/bin/dot/ -Tpdf /Users/yashjain/Downloads/graphdata/dot/7dfs.dot -o /Users/yashjain/Downloads/graphdata/dot/7dfs.pdf
DFS traversal is in
/Users/yashjain/Downloads/graphdata/dot/7dfs.pdf
DFS ORDER:  0 3 1 2 4 5 6 
NOLOOP
Work Done 13
DFS ASSERT PASSED


Testing Graph ends
